## Demo

This notebook serves to demo the ongoing training code.

In [ ]:
! ls /home/jtousmayol/scripts/datasets2.py

In [ ]:
! ls ../utils

In [ ]:
!cp /home/jtousmayol/scripts/datasets2.py ../utils

In [ ]:
from morphospectro.utils.datasets2 import GalaxyDataset2
import torch
import torch.nn as nn
import torch.nn.functional as F

import torchvision
import torchvision.transforms as transforms
from skimage import io, transform
import matplotlib.pyplot as plt

from morphospectro.utils.networks import Feedforward,FullNet,ConvNet,ConvNetBasic,SpectraEmbedding,ImageEmbedding

In [ ]:
g = GalaxyDataset2('/home/jtousmayol/scripts/spectra/sample_spectra.h5', '/home/jtousmayol/scripts/img')

In [ ]:
dataset = GalaxyDataset2('/home/jtousmayol/scripts/spectra/sample_spectra.h5', '/home/jtousmayol/scripts/img',size=5000)
#dataset= GalaxyDataset2(spectra_file = "/home/drd13/outputs/data/raw/data/s0_spectra.h5",image_folder = "/home/drd13/outputs/data/raw/data/images2",debug=False,size=1000)


n_batch = 100
lr = 0.001

loader = torch.utils.data.DataLoader(dataset = dataset,
                                     batch_size = n_batch,
                                     shuffle = True,
                                     drop_last=True)

In [ ]:
test_dataset = GalaxyDataset2('/home/jtousmayol/scripts/spectra/sample_spectra.h5', '/home/jtousmayol/scripts/img',size=500,idx_start=5000)

#test_dataset= GalaxyDataset(spectra_file = "/home/drd13/outputs/data/raw/data/s0_spectra.h5",image_folder = "/home/drd13/outputs/data/raw/data/images2",debug=False,size=500,idx_start=3000)



test_loader = torch.utils.data.DataLoader(dataset = test_dataset,
                                     batch_size = n_batch,
                                     shuffle = True,
                                     drop_last=True)

In [ ]:
spec_network = Feedforward([1000,256,50],final_activation=True)
im_network = ConvNetBasic(n_channels = 1)
merged_network = Feedforward([100,32,1])

full_network = FullNet(spec_network,im_network,merged_network)
optimizer = torch.optim.Adam(full_network.parameters(), lr=lr)

In [ ]:
#loss = torch.nn.CrossEntropyLoss()
loss = torch.nn.BCEWithLogitsLoss()

In [ ]:
full_network

In [ ]:
dataset[0][0].shape

In [ ]:
label_real = torch.ones(n_batch,1)
label_scrambled = torch.zeros(n_batch,1)

for i in range(1000):
    for j,(spec,im,idx) in enumerate(loader):
        
        optimizer.zero_grad()
        #scrambled_spec = spec[torch.randperm(n_batch)] #shuffle spec within batch
        
        prob_real = full_network(spec,im)
        err_real = loss(prob_real,label_real)
        #err_real.backward()
        
        (scrambled_spec,scrambled_im,scrambled_idx) = next(iter(loader))


        prob_scrambled = full_network(scrambled_spec,im)
        err_fake = loss(prob_scrambled,label_scrambled)
        #err_fake.backward()
        
        err_tot = err_fake+err_real#err_real+err_fake
        err_tot.backward()
        
        optimizer.step()
        print(f"err:{err_tot},j:{j}")
    """    
    for (spec,im,idx) in test_loader:
        with torch.no_grad():
            prob_real = full_network(spec,im)
            err_real = loss(prob_real,label_real)
            (scrambled_spec,scrambled_im,scrambled_idx) = next(iter(loader))
            prob_scrambled = full_network(scrambled_spec,im)
            err_fake = loss(prob_scrambled,label_scrambled)
            err_tot = err_fake+err_real
            print(f"test err:{err_tot}")
   """ 
    

In [ ]:
for (spec,im,idx) in test_loader:
    with torch.no_grad():
        prob_real = full_network(spec,im)
        err_real = loss(prob_real,label_real)
        (scrambled_spec,scrambled_im,scrambled_idx) = next(iter(test_loader))
        prob_scrambled = full_network(scrambled_spec,im)
        err_fake = loss(prob_scrambled,label_scrambled)
        err_tot = err_fake+err_real
        print(f"test err:{err_tot}")

In [ ]:
print(torch.mean(F.sigmoid(prob_real)))

In [ ]:
torch.sum(prob_real>0)

In [ ]:
torch.sum(prob_scrambled>0)

In [ ]:
plt.imshow(dataset[400][1][0])

In [ ]:
! pip install captum

In [ ]:
from captum.attr import Saliency
saliency = Saliency(full_network)

In [ ]:
sal_inputs = saliency.attribute((spec,im))

In [ ]:
idx = 5
fig,ax = plt.subplots(2,1)
ax[0].plot(sal_inputs[0][idx]*20,label="explanation")
ax[0].legend()
ax[1].plot(spec[idx].detach().numpy(),label="spectra")
ax[1].legend()
ax[1].set_xlabel("wavelength-ish")
ax[0].set_ylabel("importance")
ax[0].set_ylabel("flux-ish")
ax[0].set_xlabel("wavelength-ish")

In [ ]:
idx = 10
fig,ax = plt.subplots(2,1,figsize= (8,8))
ax[0].imshow(sal_inputs[1][idx,0])
ax[0].axis("off")
ax[0].set_title("attribution")
ax[1].set_title("input image")
ax[1].imshow(im[idx,0])
ax[1].axis("off")


In [ ]:
plt.imshow(dataset.get_image(0))
plt.axis("off")

In [ ]:
plt.imshow(dataset[0][1][0])
plt.axis("off")

In [ ]:
plt.plot(dataset[0][0])
plt.ylabel("flux")
plt.xlabel("wavelength-ish")